In [1]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM

# Part 0 - Set up

In [2]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')

In [3]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 50

#initialize theta1 and theta2 based on estimates
theta1 = np.array([3.79292846,  1.95860809,  0.75734848, -1.47436869 ])
theta2 =  np.array([2.47702906, 1.15401209, 1.00765258])
xi = np.genfromtxt('xi.csv', delimiter=',')

v0 = np.genfromtxt('simulations.csv', delimiter=',')
v = np.tile(v0.reshape(NSIM,3,1) , (1,1,NOBS))

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [61]:
def cal_same_mkt(data):
    same_mkt = np.array([data['Market_ID']],dtype=np.float32 )
    same_mkt = (same_mkt.transpose()).dot( 1/same_mkt)
    same_mkt = np.equal(same_mkt,np.ones((NOBS,NOBS)) )
    same_mkt = same_mkt.astype(np.float32)
    return same_mkt

In [62]:
def cal_sim_s(data, v, xi, theta1, theta2):
    """calculate market share across each simulated consumer
    
    Note: we are not summing accross consumers when this is called
    
    Also note: this function takes different arguments than the 
    one from estimation"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO', 'Premium']])
    delta = xi + np.matmul(np.array(x),theta1)
    delta  = np.tile( delta  ,(NSIM,1))
    
    
    x = (x.transpose()[:-1])
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1, NOBS))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() , 
                           index= data.index ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    return sim_exp/sum_exp


def cal_s(data, v, xi, theta1, theta2):
    """Calculate market share
    Calculates choice probability in each simulation, 
    then takes the sum"""
    
    shares = (1./NSIM)*cal_sim_s(data, v, xi, theta1, theta2)
    shares = (1./NSIM)*cal_sim_s(data, v, xi, theta1, theta2).sum(axis=0)
    return shares

sim_s = cal_s(data, v, xi, theta1, theta2)

# Part 2 - Elasticities

In [70]:

def cal_price_deriv(data, v, xi, theta1, theta2):
    """calculate own price derivative""" 
    alpha = abs(theta1[-1])
    same_mkt = cal_same_mkt(data)
    sim_shares = cal_sim_s(data, v, xi, theta1, theta2)
    cross_deriv = np.zeros((NOBS,NOBS))
    for sim_share in sim_shares:
        sim_share = sim_share.reshape((NOBS,1))
        cross_deriv = cross_deriv + sim_share.dot(sim_share.transpose())*same_mkt
        
    own = np.identity(NOBS)
    cross = (1 - own)
    own_deriv  = -(1-sim_shares) * sim_shares

    own_deriv = own_deriv.sum(axis=0)
    
    sim_deriv = 1./(NSIM) * alpha * (cross_deriv*cross +own_deriv*own )
    return sim_deriv


def cal_price_elast(data, v, xi, theta1, theta2):
    """calculate the elasticity using the price derivative matrix"""
    
    share = np.array([data['Inside Good Share']])
    price = np.array([data['Premium']]).transpose()
    deriv = cal_price_deriv(data, v, xi, theta1, theta2)
    return (price).dot(1/share)*deriv

elast = cal_price_elast(data, v, xi, theta1, theta2)
print elast

yo
[[-2.92605277  0.76400122  0.57909378 ...  0.          0.
   0.        ]
 [ 1.13900628 -2.44754321  0.87149389 ...  0.          0.
   0.        ]
 [ 1.26016957  1.27207397 -2.13299913 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... -3.56039465  0.30233522
   0.35700305]
 [ 0.          0.          0.         ...  0.32592397 -2.96134151
   0.32779493]
 [ 0.          0.          0.         ...  0.51562516  0.43917422
  -3.20069863]]


In [6]:
def comp_avg_elasticity(data, v, xi, theta1, theta2):
    """figure out what the average elasticty between goods is"""
    elast = cal_price_elast(data, v, xi, theta1, theta2)
    
    #who cares about run time?
    avg_elasticity = np.zeros((NPLANS,NPLANS))
    
    elast = elast
    
    
    plan_mkt = np.array(data[['Plan_ID']])
    for plan_j in range(1,NPLANS+1):
        for plan_k in range(1,NPLANS+1):
            data_k = data[ (data['Plan_ID'] == plan_k) ].index.values
            data_j = data[ (data['Plan_ID'] == plan_j)].index.values
            if len(data_k) >0 and len(data_j) > 0:
                lenobs = 0
                for ind_j in data_j:
                    for ind_k in data_k:
                        avg_elasticity[plan_j-1][plan_k-1] = (avg_elasticity[plan_j-1][plan_k-1]
                                                               + elast[ind_j][ind_k]) 
                        lenobs = lenobs + same_mkt[ind_j][ind_k]
                avg_elasticity[plan_j-1][plan_k-1] =  (avg_elasticity[plan_j-1][plan_k-1]
                                                        /lenobs)
                
    return avg_elasticity
                
avg_elasticity = comp_avg_elasticity(data, v, xi, theta1, theta2)
np.savetxt("elasticity.csv", avg_elasticity, delimiter=",")

Below are the average cross price elasticities calculated using the random coefficients among the 16 plans

In [7]:
print pd.DataFrame(avg_elasticity)

           0          1          2          3          4          5   \
0  -44.142074   4.229599  12.128709   8.949837   5.220670   6.673555   
1    4.229599 -28.811290   5.734812   4.271177   4.247828   5.590400   
2   12.128709   5.734812 -46.457238   8.876038   5.673209   7.346797   
3    8.949837   4.271177   8.876038 -38.343670   4.589591   6.329595   
4    5.220670   4.247828   5.673209   4.589591 -30.003201   5.425510   
5    6.673555   5.590400   7.346797   6.329595   5.425510 -36.514769   
6    9.405518   5.198366  10.349210   9.750692   5.324438   6.272727   
7    7.356074   6.269100   6.881948   5.285341   5.197053   7.142302   
8    9.438004   4.175607  10.063152   9.050425   4.190293   5.710544   
9    4.084806   3.275628   4.203350   3.465120   3.761349   4.736849   
10   5.557813   4.853855   5.989162   4.960306   3.985310   5.820759   
11  11.020038   4.929080  10.801377   9.730867   4.980816   6.479405   
12   7.517452   3.964241   9.986908   6.816183   4.595642   6.29

## Markups

How does the markup vary with market structure?

In [8]:
def comp_markup(data, v, xi, theta1, theta2):
    
    shares = np.array(data['Inside Good Share'])
    #caclulate formula
    own_deriv  = cal_price_deriv(data, v, xi, theta1, theta2)
    own_deriv = np.diag(own_deriv)
    
    #take inverse and calc markup
    inv_deriv = 1/own_deriv
    markup = - inv_deriv*shares
    return markup


data['Markup'] = comp_markup(data, v, xi, theta1, theta2)
data['Marginal Cost'] = data['Premium'] - data['Markup']
data['Unobs'] = xi

Below we calculate the average markup per plan

In [9]:
print data[['Plan_ID','Markup']].groupby('Plan_ID').mean()

           Markup
Plan_ID          
1        0.892770
2        0.786449
3        0.913874
4        0.849613
5        0.792455
6        0.836131
7        0.893042
8        0.846347
9        0.849672
10       0.764218
11       0.800794
12       0.885903
13       0.837991
14       0.821563
15       0.917050
16       0.821938


As we can see the more plans in a market, the lower the markup. This is the same as the previous homework

In [10]:
mean_markup = data[['Market_ID','Markup']].groupby('Market_ID').mean()
no_firms = data[['Market_ID','Plan_ID']].groupby('Market_ID').count()

model_q2 = sm.OLS(mean_markup,sm.add_constant(no_firms))
result_q2 = model_q2.fit()
print result_q2.summary()

                            OLS Regression Results                            
Dep. Variable:                 Markup   R-squared:                       0.858
Model:                            OLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     3621.
Date:                Sun, 11 Nov 2018   Prob (F-statistic):          6.89e-256
Time:                        10:42:37   Log-Likelihood:                 1251.3
No. Observations:                 600   AIC:                            -2499.
Df Residuals:                     598   BIC:                            -2490.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1051      0.004    266.550      0.0

# Part 3 - Marginal Costs

The following regression will show the relationship between Marginal costs and the plan characteristics

In [11]:
model_q3 = sm.OLS(data['Marginal Cost'], 
                   sm.add_constant(data[['Network Score','Satisfaction Score','PPO']]))
result_q3 = model_q3.fit()
print result_q3.summary()

                            OLS Regression Results                            
Dep. Variable:          Marginal Cost   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     1258.
Date:                Sun, 11 Nov 2018   Prob (F-statistic):               0.00
Time:                        10:42:37   Log-Likelihood:                 4406.3
No. Observations:                3300   AIC:                            -8805.
Df Residuals:                    3296   BIC:                            -8780.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  1.4380      0

# Part 4 - Counterfactuals

In [12]:
def cal_mkt_sim_s(p, data, v, xi, theta1, theta2,nobs):
    """only calculate sims within the same market
    will use this when recalculating the FOCs"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']])
    delta = xi + np.matmul(np.array(x),theta1[:-1]) + p*theta1[-1]
    delta  = np.tile( delta  ,(NSIM,1))
    
    x = x.transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1, nobs))
    
    #add to calcualte market shares
    sim_exp = np.exp(delta + (theta2*v*x).sum(axis=1)).transpose()
    
    return  (1./ (sim_exp.sum(axis=0) +1) ) * sim_exp


def cal_mkt_s(p, data, v, xi, theta1, theta2,nobs):
    """calc market share within the same market"""
    shares = (1./NSIM)*cal_mkt_sim_s(p, data, v, xi, theta1, theta2,nobs).sum(axis=1)

    return shares


def cal_mkt_deriv(p, data, v, xi, theta1, theta2 , nobs):
    """calculate price derivative, but only in the same market"""
    alpha = abs(theta1[-1])
    sim_shares = cal_mkt_sim_s(p, data, v, xi, theta1, theta2, nobs)
    own_deriv  = -(1-sim_shares) * sim_shares
    
    own_deriv = own_deriv.sum(axis=1)
    sim_deriv = 1./(NSIM) * alpha * (own_deriv)

    return sim_deriv


def comp_foc(p, data, v, xi, theta1, theta2, subs, nobs):
    """compute the first order condition (market by market)"""
    shares =  cal_mkt_s(p, data, v, xi, theta1, theta2,  nobs)
    #caclulate formula
    own_deriv  = cal_mkt_deriv(p, data, v, xi, theta1, theta2 , nobs)
    inv_deriv = 1/own_deriv
    markup = - inv_deriv*shares
    return markup - (p - data['Marginal Cost'] + subs)

In [13]:
#numerically solve on a market by market basis
new_prices = [[]] * NMKTS


for i in range(1,NMKTS+1):
    #set up mkt level variables
    mkt_data = data.copy()[data['Market_ID'] == i]
    mkt_data['Marginal Costs'] = (mkt_data.copy()['Marginal Cost'] ) #apply subsidy
    mkt_obs = mkt_data['Plan_ID'].count()
    mkt_prices = np.array(mkt_data['Premium']).squeeze()
    mkt_v = np.tile(v0.reshape(NSIM,3,1) , (1,1,mkt_obs))
    mkt_xi = mkt_data['Unobs']
    
    #calculate FOCs
    mkt_new_prices = fsolve(comp_foc, mkt_prices, args= (mkt_data, mkt_v, mkt_xi,
                                                         theta1, theta2, .25, mkt_obs) )
    new_prices[i-1] = mkt_new_prices
    

#flatten result to 1d array
new_prices = np.array([ p for  mkt_new_prices in new_prices for p in  mkt_new_prices ])

#write to file
np.savetxt('prices_blp.csv', new_prices, delimiter=',')

In [14]:
#avoid caclulating everytime
new_prices = np.genfromtxt('prices_blp.csv', delimiter=',')

# Part 1 - Uninsurance rate

Below we calcualte how much the uninsurance rate delcined after the the subsidy

In [15]:
#outside good shares

cf_data = data.copy()
data = comp_outside_good(data,'Inside Good Share')

cf_data['Premium'] = new_prices
cf_data['New Inside Good'] =  cal_s(cf_data, v, xi, theta1, theta2)
cf_data = comp_outside_good(cf_data,'New Inside Good')

#compare the mean outside good before and after the rebate. It decreases.
print 'Outside Good (Before Rebate): %s'%data['Outside Good Share'].mean()
print 'Outside Good (After Rebate): %s'%cf_data['Outside Good Share'].mean()

Outside Good (Before Rebate): 0.134505918182
Outside Good (After Rebate): 0.107288544577


# Part 2 - Change in Profits
Below we ecalculate the change in profits per enrollee after the rebate

In [16]:
#profits per enrollee, comparision
print 'Per Enrollee (Before Rebate): %s'%(data['Premium'] - cf_data['Marginal Cost']).mean()
print 'Per Enrollee (After Rebate): %s'%(cf_data['Premium'] - cf_data['Marginal Cost'] + .25).mean()

Per Enrollee (Before Rebate): 0.843850773635
Per Enrollee (After Rebate): 0.847090861972


# Part 3 - Change in Consumer Surplus

Below we calculate the change in consumer surplus

In [34]:
def comp_sim_exp(data, v, xi, theta1, theta2):
    """simulate utility in every market"""
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO', 'Premium']])
    delta = xi + np.matmul(np.array(x),theta1)
    delta  = np.tile( delta  ,(NSIM,1))
    
    x = (x.transpose()[:-1])
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1, NOBS))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() , 
                           index= data.index )
    return sim_exp


def comp_exp(data, v, xi, theta1, theta2):
    """Calculate market share
    Calculates individual choice probability first, then take sum"""
    
    shares = (1./NSIM)*comp_sim_exp(data, v, xi, theta1, theta2 ).sum(axis=1)
    return shares

In [36]:
def comp_surplus(data, cf_data, v, xi, theta1, theta2 ):
    """ compute exp(delta_j) to compute the change in consumer surplus """
    
    alpha = abs(theta1[-1])
    exp = comp_exp(data, v, xi, theta1, theta2)
    
    cf_exp = comp_exp(cf_data, v, xi, theta1, theta2 )
    
    utility_ratio = cf_exp.sum()/exp.sum()
    return 1/alpha * np.log( utility_ratio )


print 'Change in consumer surplus: %s'%comp_surplus(data, cf_data, v, xi, theta1, theta2 )

Change in consumer surplus: 0.24700632946140574


## Surplus and market structure

In [48]:
def sum_by_mkt(inner, data):
    """function for summing utility by market"""
    inner['mkt_id'] = data['Market_ID']
    inner = inner.groupby('mkt_id').sum()
    return inner


def comp_surplus_mkt(data, cf_data, v, xi, theta1, theta2 ):
    """compute the change in consumer surplus on a 
    per market basis """
    alpha = abs(theta1[-1])
    
    #compute surplus change by market
    exp = comp_exp(data, v, xi, theta1, theta2 )
    exp = sum_by_mkt(exp, data)
    cf_exp = comp_exp(cf_data, v, xi, theta1, theta2 )
    cf_exp = sum_by_mkt(cf_exp, data)
    
    utility_ratio = np.array(cf_exp/exp)
    return 1/alpha * np.log( utility_ratio )
    
    

mkt_surplus = comp_surplus_mkt(data, cf_data, v, xi, theta1, theta2 )
no_firms = data[['Market_ID','Plan_ID']].groupby('Market_ID').count()

model_q4 = sm.OLS(mkt_surplus,sm.add_constant(no_firms))
result_q4 = model_q4.fit()
print result_q4.summary()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     3643.
Date:                Sun, 11 Nov 2018   Prob (F-statistic):          1.48e-256
Time:                        11:07:26   Log-Likelihood:                 3163.4
No. Observations:                 600   AIC:                            -6323.
Df Residuals:                     598   BIC:                            -6314.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2360      0.000   1378.319      0.0

Using the table above, we can see that surplus is increasing with the number of firms in each market. This is because when there are more firms, the rebate is passed more directly to consumers.

# Part 4

The answers are roughly the same between BLP and the logit model.